In [1]:
import os
if os.path.basename(os.getcwd())!="HUST-CV-Neural-Style-Transfer":
    %cd ../../

e:\pyenv\GTCC\KPG-RL\HUST-CV-Neural-Style-Transfer


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import cv2
import torchvision.transforms.v2 as transforms
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
import math
import torchvision.models as models
import albumentations as A
from albumentations.pytorch import ToTensorV2
import os
import pandas as pd
from sklearn.model_selection import train_test_split

c:\Users\tungv\AppData\Local\Programs\Python\Python311\Lib\site-packages\albumentations\__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.6 (you have 1.4.23). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [3]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")
DATASET_PATH = "./datasets" 

Using device: cuda


In [4]:
class Failure(Dataset):
    def __init__(self, root_dir, stylization_list):
        super(Failure, self).__init__()
        self.root_dir = root_dir
        self.df = pd.read_csv(os.path.join(self.root_dir,"user-ratings-all.csv"))
        self.df.set_index("name",inplace=True)
        self.stylization = stylization_list
        self.size = [150,300,500,700]

        self.transforms = transforms.Compose([
            transforms.Resize([456,456]),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # Normalize the image
        ])
    
    def __len__(self):
        return len(self.stylization)
    
    def __getitem__(self, index):
        name = self.stylization[index]
        img = Image.open(os.path.join(self.root_dir,"stylizations",name))
        img = self.transforms(img)
        target = self.df.at[name,"mean_rating"]
        return img, torch.Tensor([target/10])

In [5]:
stylization_list = os.listdir(os.path.join(DATASET_PATH,"stylizations"))

train_list,test_list = train_test_split(stylization_list,test_size=0.2,random_state=42)


train_dataset = Failure(DATASET_PATH,train_list)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataset = Failure(DATASET_PATH,test_list)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

c:\Users\tungv\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(
c:\Users\tungv\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [6]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.resnet = nn.Sequential(*list(models.resnet34(pretrained=True).children())[:-1])        
        for param in self.resnet.parameters():
            param.requires_grad = False
        self.resnet.eval()
        
        self.flat = nn.Flatten()
        
        self.linear = nn.Sequential(
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        x = self.resnet(x)
        x = self.flat(x)
        x = self.linear(x)
        
        return x

In [7]:
model = Model().to(DEVICE)
optim = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()
def train(model, train_loader, criterion, optimizer, epoch):
    model.train()
    running_loss = 0.0
    for i, (images, labels) in enumerate(tqdm(train_loader)):
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}], Loss: {running_loss/len(train_loader):.4f}")
    
    
def test(model, test_loader):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for i, (images, labels) in enumerate(tqdm(test_loader)):
            images = images.to(DEVICE)
            labels = labels.to(DEVICE)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
    print(f"Test Loss: {running_loss/len(test_loader):.4f}")
    
for k in range(10):
    print(f"Epoch {k+1}")
    train(model, train_loader, criterion, optim, k)
    test(model, test_loader)
    

c:\Users\tungv\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\tungv\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1


  0%|          | 0/500 [00:00<?, ?it/s]

Epoch [1], Loss: 0.0227


  0%|          | 0/125 [00:00<?, ?it/s]

...

Test Loss: 0.0119
Epoch 10


  0%|          | 0/500 [00:00<?, ?it/s]

Epoch [10], Loss: 0.0119


  0%|          | 0/125 [00:00<?, ?it/s]

Test Loss: 0.0176


In [10]:
#save model
torch.save(model.state_dict(), "models/model_rating.pth")

In [11]:
transforms = transforms.Compose([
            transforms.Resize([456,456]),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # Normalize the image
        ])

def predict(model, image_path):
    image = Image.open(image_path)
    image = transforms(image)
    output = model(image.unsqueeze(0).to(DEVICE))
    output = output.cpu().detach().numpy()
    return output[0][0] * 10

predict(model, os.path.join(DATASET_PATH,"stylizations","stylization_0.png"))

c:\Users\tungv\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(
